In [1]:
!export PYTHONPATH=${PYTHONPATH}:/home/sai/PythonRobotics/PathPlanning/CubicSpline

In [1]:
from ppo_control import PPOControl
import cubic_spline_planner
import matplotlib.pyplot as plt
import time
import numpy as np
import math
%matplotlib tk

In [70]:
husky_ppo = PPOControl()

In [5]:
ax = [0.0,6.0, 12.5, 14.0, 17.5, 20.0, 25.0] 
ay = [0.0,-3.0, -5.0, -4, 3.0, 2, 0.0]
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(ax, ay, ds= 0.5)
plt.plot(cx, cy, "+")
plt.show()
cv = [1.0]*len(cx)
husky_ppo.set_waypoints_from_list(cx, cy, cyaw, cv)

In [71]:
#husky_ppo.read_waypoint_file("./unity_waypoints_bkp.txt")
#husky_ppo.read_waypoint_file("ww.txt")
husky_ppo.read_waypoint_file("./unity_flat.txt")
cx = [i[0] for i in husky_ppo.waypoints_list]
cy = [i[1] for i in husky_ppo.waypoints_list]
plt.plot(cx, cy, "+")
plt.show()

In [15]:
def simulate_warthog(X, v, w, dt):
    xcurr = X[0] + v*math.cos(X[2])*dt
    ycurr = X[1] + v*math.sin(X[2])*dt
    thcurr = X[2]+w*dt
    return [xcurr, ycurr, thcurr]

In [66]:
def simulate_warthog2(X, v, w, dt, v_prev, w_prev):
    xcurr = X[0] + v_prev*math.cos(X[2])*dt
    ycurr = X[1] + v_prev*math.sin(X[2])*dt
    thcurr = X[2]+w_prev*dt
    alpha = 0.8
    beta = 0.2
    vcurr = v*(alpha) + v_prev*(1-alpha)
    wcurr = w*(alpha) + w_prev*(1-alpha)
    vcurr = v*(alpha)
    wcurr = w*(beta)
    return [xcurr, ycurr, thcurr, vcurr, wcurr]

In [72]:
#husky_ppo.set_pose([30.67, -33.33, 3.06])
#husky_ppo.set_pose([120.180, -78.957, 140*math.pi/180.])
#husky_ppo.set_pose([132.180, -78.957, 160*math.pi/180.])
husky_ppo.set_twist([0. , 0.])
#husky_ppo.set_pose([0. , 0.5, 7*math.pi/4])
start_idx = 2
husky_ppo.set_pose([cx[start_idx]+0.05 , cy[start_idx] + 0.05, -1*husky_ppo.waypoints_list[start_idx][2]])
husky_ppo.read_ppo_policy("./model1.zip")
#husky_ppo.read_tf_frozen_graph("/home/sai/hdd1/ml-master/ml-agents/config/ppo/results/wlong_path52/3DBall/frozen_graph_def.pb")

Using cpu device
Wrapping the env in a DummyVecEnv.


In [73]:
x_pose = []
y_pose = []
v_prev = 0
w_prev = 0
for i in range(0, 2000):
    obs = husky_ppo.get_observation()
    twist = husky_ppo.get_ppo_control(np.array(obs).reshape(1,42))[0]
    print(twist)
    v = np.clip(twist[0][0], 0, 1) * 4.0
    w = np.clip(twist[0][1], -1, 1) * 2.5
    current_pose = simulate_warthog2(husky_ppo.get_pose(), v, w, 0.05, v_prev, w_prev)
    #current_pose = simulate_warthog(husky_ppo.get_pose(), v, w, 0.05)
    print(twist[0], current_pose)
    husky_ppo.set_pose(current_pose[0:3])
    #husky_ppo.set_pose(current_pose)
    v_prev = current_pose[3]
    w_prev = current_pose[4]
    husky_ppo.set_twist([v, w])
    x_pose.append(current_pose[0])
    y_pose.append(current_pose[1])
    print(husky_ppo.closest_idx, husky_ppo.closest_dist)

[[1.         0.53480256]]
[1.         0.53480256] [1.232624, 0.044218099000000004, -6.194896087531383, 3.2, 0.6685031950473785]
2 0.07071067811865478
[[ 1.         -0.34642062]]
[ 1.         -0.34642062] [1.392000806069018, 0.0583260289416522, -6.161470927779014, 3.2, -0.4330257698893547]
2 0.07071067811865478
[[ 1. -1.]]
[ 1. -1.] [1.5508171172421823, 0.07775228193466882, -6.183122216273482, 3.2, -1.25]
2 0.21897139904806528
[[ 1. -1.]]
[ 1. -1.] [1.7100167755969065, 0.09373567268066246, -6.245622216273482, 3.2, -1.25]
3 0.20989297774049037
[[ 1. -1.]]
[ 1. -1.] [1.8699039100049482, 0.09974435396567236, -6.308122216273482, 3.2, -1.25]
3 0.14673723939494485
[[ 1. -1.]]
[ 1. -1.] [2.0298541646280603, 0.09575486201786752, -6.370622216273482, 3.2, -1.25]
3 0.21743052600570656
[[ 1. -1.]]
[ 1. -1.] [2.1892429371449067, 0.08178277571792822, -6.433122216273482, 3.2, -1.25]
3 0.3481194911707657
[[ 1.        -0.5731517]]
[ 1.        -0.5731517] [2.3474478178097606, 0.05788265576385028, -6.4956

[[1.         0.36872634]]
[1.         0.36872634] [1.1199378328084566, -16.570476652778193, -8.581880870523314, 3.2, 0.4609079286456108]
41 0.445157992944589
[[1.         0.41506916]]
[1.         0.41506916] [1.0134893960901237, -16.68992844355683, -8.558835474091033, 3.2, 0.5188364535570145]
41 0.4175947127951447
[[1.        0.4533406]]
[1.        0.4533406] [0.9098217952007662, -16.81180144518494, -8.532893651413183, 3.2, 0.5666757375001907]
41 0.45223409315199564
[[1.         0.41613844]]
[1.         0.41613844] [0.8093503285648932, -16.936322465065267, -8.504559864538173, 3.2, 0.5201730504631996]
41 0.5375023054406572
[[1.         0.44225386]]
[1.         0.44225386] [0.7124468686414123, -17.063639861578306, -8.478551212015013, 3.2, 0.5528173223137856]
42 0.6151411343002169
[[1.         0.46378744]]
[1.         0.46378744] [0.6188871626545596, -17.19343424287293, -8.450910345899324, 3.2, 0.5797342956066132]
42 0.5720541996526343
[[1.         0.49823242]]
[1.         0.49823242] [0.

[[1.         0.17566341]]
[1.         0.17566341] [1.5287842556756093, -36.217550068848226, -8.386869744704022, 3.2, 0.21957926452159882]
82 0.3964503523867989
[[1.         0.26756853]]
[1.         0.26756853] [1.4475005577993247, -36.355365018906475, -8.375890781477942, 3.2, 0.33446066081523895]
83 0.42172416750929287
[[1.       0.315832]]
[1.       0.315832] [1.3677347936181175, -36.49406405590597, -8.35916774843718, 3.2, 0.3947899863123894]
83 0.37572877167185215
[[1.         0.25846946]]
[1.         0.25846946] [1.2902995432867752, -36.63407756241917, -8.33942824912156, 3.2, 0.32308682799339294]
83 0.3962423592888322
[[1.         0.05892915]]
[1.         0.05892915] [1.2156429957392705, -36.7755922256463, -8.323273907721891, 3.2, 0.07366143632680178]
83 0.47543878704123466
[[1.         0.24212888]]
[1.         0.24212888] [1.1432821660227623, -36.918294399150206, -8.31959083590555, 3.2, 0.30266109853982925]
84 0.46560595684312744
[[1.         0.37095183]]
[1.         0.37095183] [1

[[ 1.       -0.401629]]
[ 1.       -0.401629] [16.79305344400004, -50.39986240976797, -7.254128942135871, 3.2, -0.5020362511277199]
127 0.3191546921146582
[[ 1.        -0.4098769]]
[ 1.        -0.4098769] [16.883376786175596, -50.53192941495184, -7.279230754692257, 3.2, -0.5123461410403252]
127 0.1845880671186594
[[ 1.         -0.38999033]]
[ 1.         -0.38999033] [16.970356900355046, -50.66622185605457, -7.3048480617442735, 3.2, -0.48748791217803955]
127 0.13472632021096248
[[ 1.         -0.35400823]]
[ 1.         -0.35400823] [17.053868641461104, -50.802698187692826, -7.3292224573531755, 3.2, -0.44251028448343277]
127 0.2334117092754799
[[ 1.         -0.43669692]]
[ 1.         -0.43669692] [17.134029377446126, -50.94116932694697, -7.351347971577347, 3.2, -0.5458711460232735]
128 0.20913317515359559
[[ 1.         -0.41821498]]
[ 1.         -0.41821498] [17.211106998155977, -51.08138002691565, -7.378641528878511, 3.2, -0.5227687209844589]
128 0.1311920872273278
[[ 1.        -0.411410

[[1.         0.14830267]]
[1.         0.14830267] [3.87495694573215, -69.39556488054575, -8.466616232691017, 3.2, 0.18537834286689758]
165 0.28602891497362914
[[1.         0.11726448]]
[1.         0.11726448] [3.7829529596439646, -69.52646662439923, -8.457347315547672, 3.2, 0.14658059924840927]
165 0.221284672777151
[[ 1.        -0.0613363]]
[ 1.        -0.0613363] [3.69216622573318, -69.65821551033689, -8.450018285585251, 3.2, -0.0766703812405467]
165 0.2602111257033309
[[1.         0.01633647]]
[1.         0.01633647] [3.602347512989839, -69.79062623060202, -8.453851804647279, 3.2, 0.020420593209564686]
166 0.29512240095212827
[[1.         0.03701461]]
[1.         0.03701461] [3.512021862450795, -69.92269165701899, -8.4528307749868, 3.2, 0.046268259175121784]
166 0.2182828574635349
[[ 1.         -0.05952727]]
[ 1.         -0.05952727] [3.4218311016881295, -70.05484923974905, -8.450517362028044, 3.2, -0.07440909277647734]
166 0.24317735448691624
[[ 1.       -0.161807]]
[ 1.       -0.1

...

[1.         0.78438455] [11.35889542289514, -97.82657780700417, -4.823233634161447, 3.2, 0.9804806858301163]
300 0.40791008051799177
[[1.        0.8688471]]
[1.        0.8688471] [11.376594272551028, -97.6675597240046, -4.774209599869941, 3.2, 1.0860588401556015]
300 0.44558751275286884
[[1.        0.7165312]]
[1.        0.7165312] [11.386479272462816, -97.50786536976268, -4.719906657862161, 3.2, 0.8956640213727951]
301 0.38237196755213887
[[1.         0.74692345]]
[1.         0.74692345] [11.387682089529507, -97.34786989097937, -4.675123456793521, 3.2, 0.9336543083190918]
301 0.3437392658012862
[[1.        0.8162887]]
[1.        0.8162887] [11.381720985695097, -97.18798097566291, -4.628440741377567, 3.2, 1.0203608870506287]
301 0.38376734721111005
[[1.         0.66202724]]
[1.         0.66202724] [11.368305038135844, -97.02854442919038, -4.577422697025035, 3.2, 0.8275340497493744]
302 0.37933926674872986
[[1.        0.6576947]]
[1.        0.6576947] [11.346775933965116, -96.8699994902

[[1.         0.03758982]]
[1.         0.03758982] [-1.8736852140489422, -81.77539269626755, -5.534135458482346, 3.2, 0.04698727745562792]
344 0.25268107606040435
[[ 1.         -0.12373534]]
[ 1.         -0.12373534] [-1.7565114222965483, -81.66644177829818, -5.531786094609565, 3.2, -0.1546691730618477]
344 0.15464524183306538
[[ 1.         -0.14355364]]
[ 1.         -0.14355364] [-1.6395939190301152, -81.55721587738661, -5.539519553262657, 3.2, -0.17944205552339554]
344 0.18566865379081854
[[1.         0.00322965]]
[1.         0.00322965] [-1.5218352263838426, -81.44889741032655, -5.548491656038827, 3.2, 0.004037063918076456]
344 0.3100317515914954
[[ 1.         -0.33694372]]
[ 1.         -0.33694372] [-1.403109442032003, -81.34163983189947, -5.548289802842923, 3.2, -0.4211796447634697]
345 0.20599869595436077
[[ 1.        -0.4655003]]
[ 1.        -0.4655003] [-1.2844053103837307, -81.23435829047864, -5.569348785081097, 3.2, -0.5818753689527512]
345 0.18372213961803766
[[ 1.         -0

[[1.         0.54946196]]
[1.         0.54946196] [1.808899402499438, -79.23584768603624, -1.3956883107173823, 3.2, 0.6868274509906769]
377 1.3701022984299849
[[-1.         -0.91242635]]
[-1.         -0.91242635] [1.8367737229595733, -79.3934009223054, -1.3613469381678485, 0.0, -1.1405329406261444]
377 1.3701022984299849
[[1.         0.25459683]]
[1.         0.25459683] [1.8367737229595733, -79.3934009223054, -1.4183735851991557, 3.2, 0.3182460367679596]
377 1.2623913138242522
[[-1. -1.]]
[-1. -1.] [1.8610670394992084, -79.551545902534, -1.4024612833607577, 0.0, -1.25]
377 1.2623913138242522
[[1.         0.32053673]]
[1.         0.32053673] [1.8610670394992084, -79.551545902534, -1.4649612833607577, 3.2, 0.4006709158420563]
377 1.169787016283109
[[-1.        -0.7561384]]
[-1.        -0.7561384] [1.8779690517674175, -79.7106506541327, -1.444927737568655, 0.0, -0.945172980427742]
377 1.169787016283109
[[1.        0.4967556]]
[1.        0.4967556] [1.8779690517674175, -79.7106506541327, -

[[1. 1.]]
[1. 1.] [-3.521152763822367, -74.01494348834, -3.18694030052648, 3.2, 1.25]
411 0.5519957507121761
[[1. 1.]]
[1. 1.] [-3.680988279285941, -74.00769035132946, -3.12444030052648, 3.2, 1.25]
411 0.5179870096255281
[[1. 1.]]
[1. 1.] [-3.8409647436057237, -74.01043459325417, -3.06194030052648, 3.2, 1.25]
411 0.5223138874158773
[[1. 1.]]
[1. 1.] [-4.000457452112074, -74.02316549790814, -2.99944030052648, 3.2, 1.25]
411 0.5640793348816302
[[1. 1.]]
[1. 1.] [-4.15884358919145, -74.04583335138113, -2.93694030052648, 3.2, 1.25]
411 0.635810078542381
[[1. 1.]]
[1. 1.] [-4.3155046603680525, -74.07834963619044, -2.87444030052648, 3.2, 1.25]
411 0.7284796355063845
[[1. 1.]]
[1. 1.] [-4.469828907511514, -74.12058737693972, -2.81194030052648, 3.2, 1.25]
412 0.7719405162453508
[[1. 1.]]
[1. 1.] [-4.621213697739308, -74.17238163615527, -2.74944030052648, 3.2, 1.25]
412 0.768145669292106
[[0.7116352 1.       ]]
[0.7116352 1.       ] [-4.769067876685261, -74.23353015836324, -2.68694030052648, 2.

[[ 0.44014886 -1.        ]]
[ 0.44014886 -1.        ] [-2.093534557553876, -82.71054335922456, -3.5218890737895023, 1.4084763526916504, -1.25]
436 0.9259900104573672
[[ 0.7622028 -1.       ]]
[ 0.7622028 -1.       ] [-2.1589269206399173, -82.6844023389642, -3.5843890737895023, 2.439048957824707, -1.25]
436 0.9801573476044604
[[ 0.42678127 -1.        ]]
[ 0.42678127 -1.        ] [-2.2691179112195843, -82.6321496343286, -3.6468890737895023, 1.3657000541687012, -1.25]
436 1.0127744848782232
[[ 0.7583104 -1.       ]]
[ 0.7583104 -1.       ] [-2.32886940717822, -82.59909512895345, -3.7093890737895023, 2.4265932083129886, -1.25]
436 1.0714743731297953
[[ 0.40378937 -1.        ]]
[ 0.40378937 -1.        ] [-2.431160994282073, -82.53384700480913, -3.7718890737895023, 1.2921259880065918, -1.25]
436 1.1052710307618343
[[ 0.76202446 -1.        ]]
[ 0.76202446 -1.        ] [-2.4833533766317477, -82.49576906963834, -3.8343890737895023, 2.43847827911377, -1.25]
436 1.1665102798670532
[[ 0.36698714 -

In [74]:
plt.cla()
plt.plot(x_pose, y_pose, '+g')
plt.plot(cx, cy, "+b")
#plt.xlim([0, 25])
#plt.ylim([-20, 7])

In [ ]:
obs = husky_ppo.get_observation()

In [ ]:
print(obs)

In [ ]:
print(husky_ppo.get_pose())

In [ ]:
husky_ppo.closest_idx

In [ ]:
2*math.pi - 6.1321

In [ ]:
i = 0
while i < len(obs)-3:
    print([obs[i], obs[i+1], obs[i+2], obs[i+3]])
    i = i + 4

In [ ]:
obs2 = [ 3.1718987e-01,  2.6397274e+00, -7.1165800e-02,  2.9253261e+00,
         8.2946199e-01,  2.4810174e+00, -2.3757100e-01,  2.9868939e+00,
         1.4194126e+00,  2.3704438e+00, -3.9009976e-01,  2.9857869e+00,
         2.0244434e+00,  2.2775273e+00, -4.8208833e-01,  3.2309980e+00,
         2.5908678e+00,  2.2095571e+00, -5.5408025e-01,  3.0272779e+00,
         3.1385303e+00,  2.1544523e+00, -5.6134295e-01,  2.8097680e+00,
         3.6512144e+00,  2.1172132e+00, -5.9754920e-01,  2.6893539e+00,
         4.2099533e+00,  2.0821514e+00, -6.1818290e-01,  2.6712611e+00,
         4.7472596e+00,  2.0539546e+00, -6.3812470e-01,  2.9293561e+00,
         5.2431216e+00,  2.0312028e+00, -6.4151406e-01,  2.9405370e+00,
         3.7776546e-03, -1.4553810e-03]

In [ ]:
i = 0
while i < len(obs2)-3:
    print([obs2[i], obs2[i+1], obs2[i+2], obs2[i+3]])
    i = i + 4

In [ ]:
print(husky_ppo.num_waypoints)

In [20]:
print(husky_ppo.waypoints_list)

[array([0.        , 0.        , 5.84690706, 4.        ]), array([ 0.43545074, -0.20302535,  5.84595374,  4.        ]), array([ 0.87129576, -0.40674059,  5.84405479,  4.        ]), array([ 1.30792936, -0.61183558,  5.8412255 ,  4.        ]), array([ 1.74574581, -0.81900022,  5.83748844,  4.        ]), array([ 2.1851394 , -1.02892438,  5.83287314,  4.        ]), array([ 2.62650443, -1.24229795,  5.82741558,  4.        ]), array([ 3.07023516, -1.4598108 ,  5.82115759,  4.        ]), array([ 3.5167259 , -1.68215281,  5.81414619,  4.        ]), array([ 3.96637093, -1.91001388,  5.80643284,  4.        ]), array([ 4.41956452, -2.14408387,  5.79807264,  4.        ]), array([ 4.87670098, -2.38505267,  5.78912349,  4.        ]), array([ 5.33817457, -2.63361015,  5.77964528,  4.        ]), array([ 5.8043796 , -2.89044621,  5.77000864,  4.        ]), array([ 6.27566617, -3.15603351,  5.76509605,  4.        ]), array([ 6.75167852, -3.42737363,  5.76856828,  4.        ]), array([ 7.23149358, -3.6986